### Zadanie 3.

Zadanie 3. zaczniemy od stworzenia takich samych wizualizacji wyników eksperymentu jak w poprzednich zadaniach.

Eksperyment przeprowadzamy dla różnych wartości parametrów:
- badamy $6$ wartości parametru śmiertelności równo rozłożonych w przedziale $[0.3, 0.35]$,
- badamy $6$ wartości parametru minimalnego podobieństwa równo rozłożonych w przedziale $[0.3, 0.35]$.

In [1]:
from simulator import *
from analysis  import *

In [2]:
dpr_no_steps = 6
sth_no_steps = 6

death_pr = np.linspace(.3, .35, endpoint=True, num=dpr_no_steps)
similarity_th = np.linspace(.3, .35, endpoint=True, num=sth_no_steps)

no_iters = 25

In [3]:
results = GridExpMean.RunC(
    similarity_th, 
    death_pr, 
    no_iters
)

In [4]:
labels = dict(
    x="Współczynnik śmiertelności (death_pr)",
    y="Współczynnik min. podobieństwa (similarity_th)",
    color="Populacja"
)

In [5]:
Plots.heatmap(results, death_pr, similarity_th, labels).show()

In [6]:
Plots.heatmap(np.log1p(results), death_pr, similarity_th, labels).show()

In [7]:
labels = dict(
    x="Współczynnik śmiertelności (death_pr)",
    y="log(1 + wielkość populacji)"
)

In [8]:
Plots.line(results, death_pr, similarity_th, "similarity_th", False, labels).show()

In [9]:
labels = dict(
    x="Współczynnik min. podobieństwa (similarity_th)",
    y="log(1 + wielkość populacji)"
)

In [10]:
Plots.line(results, similarity_th, death_pr, "death_pr", True, labels).show()

Wnioski:

Rozwój populacji wydaje się dużo mniej stabilny.
Spodziewalibyśmy się, że zwiększanie współczynnika min. podobieństwa i zmniejszanie parametru śmeirtelności skutkowałoby zwiększaniem się rozmiaru populacji po zakończeniu eksperymentu, ale nie zawsze tak jest (żeby zobaczyć ten efekt, musielibyśmy badać średnią z większej ilości symulacji niż poprzednio).
Zapewne jest to wynik istnienia mutantów.

W zadaniu jesteśmy proszeni o zbadanie liczności grup tego samego koloru.
To jednak nie jest możliwe - z reguły wszystkie osobniki są różnego (choć być może zbliżonego) koloru.
Wybrałem zatem $8$ kolorów (stanowiących wierzchołki "sześcianu" reprezentacji RGB - dokładniej widać to poniżej).
Stworzymy $8$ grup, po jednej dla każdego z tych kolorów, a osobników końcowej populacji przydzielimy do grupy o najbardziej podobnym (wg znormalizowanej odległości euklidesowej) do jego kolorze.

Poniżej wizualizuję wybrane kolory.

In [11]:
ColorGrouping.group_colors_plot(
    demo=True,
    title="Kolory wybrane do grupowania").show()

Badamy $6$ równo rozłożonych w przedziale $[0.3, 0.35]$ wartości współczynnika minimalnego podobieństwa przy współczynniku śmiertelności równym $0.3$.

Jak widzimy poniżej, grupy niebieska i czerwona wyróżniają się na tle pozostałych.
Dzieje się tak zapewne dlatego, że początkowa populacja jest w połowie czerwona, a w połowie niebieska.

In [12]:
sth_no_steps = 6

death_pr = .3
similarity_th = np.linspace(.3, .35, endpoint=True, num=sth_no_steps)

In [13]:
results = []

for sth in similarity_th:
    population = []
    # repeat until the resulting population is non-empty
    while len(population) == 0:
        population = Experiment.RunC(sth, death_pr)[0]
    results.append(population)

In [14]:
for i, population in enumerate(results):
    if len(population) > 0:
        group_counts = ColorGrouping.population_to_group_counts(population)

        sth = np.round(similarity_th[i], 2)

        ColorGrouping.group_colors_plot(
            group_counts,
            title=f"Liczebność osobników w poszczególnych grupach kolorystycznych (similarity_th={sth})"
        ).show()

In [15]:
for i, population in enumerate(results):
    if len(population) > 0:
        mutants = MutantAnalysis.get_mutants(population)
        mutant_counts = ColorGrouping.population_to_group_counts(mutants)

        sth = np.round(similarity_th[i], 2)

        ColorGrouping.group_colors_plot(
            mutant_counts,
            title=f"Liczebność mutantów w poszczególnych grupach kolorystycznych (similarity_th={sth})"
        ).show()

Poniżej wizualizujemy średni udział mutantów w całej populacji dla rożnych wartości parametru min. podobieństwa i współczynnika śmiertelności równego $0.3$.
Spodziewamy się, że wartości będą oscylować w okolicy $15\%$.


In [22]:
no_iter = 50
death_pr = .3

mutant_share = []

for sth in similarity_th:
    cur_sth_ms = []
    for _ in range(no_iter):
        res = []
        while len(res) == 0:
            res = Experiment.RunC(sth, death_pr)[0]
        ms = MutantAnalysis.get_mutant_share(res)
        cur_sth_ms.append(ms)
    mutant_share.append(np.mean(cur_sth_ms))

In [24]:
labels = dict(
    title=f"Udział mutantów / Współczynnik min. podobieństwa | death_pr={death_pr}",
    x="Współczynnik rozmnażania (replication_pr)",
    y="Udział mutantów w populacji"
)

Plots.single_line(
    x=similarity_th,
    y=mutant_share,
    labels=labels
).show()

Jako optymalne parametry wybieram:
- współczynnik śmiertelności równy $0.3$,
- współczynnik minimalnego podobieństwa równy $0.35$.

Taki dobór parametrów skutkuje, jak widzieliśmy powyżej, populacją liczącą średnio ok. $1000-2000$ osobników.

In [18]:
dpr = .3
sth = .35

results = []
while len(results) == 0:
    results = Experiment.RunC(sth, dpr)[0]
group_counts = ColorGrouping.population_to_group_counts(results)

ColorGrouping.group_colors_plot(
    group_counts,
    title="Liczebność osobników w poszczególnych grupach kolorystycznych"
).show()

Poniżej wizualizujemy wyłącznie mutantów.

In [19]:
mutants = MutantAnalysis.get_mutants(results)
mutant_counts = ColorGrouping.population_to_group_counts(mutants)

ColorGrouping.group_colors_plot(
    mutant_counts, 
    title="Liczebność mutantów w poszczególnych grupach kolorystycznych"
).show()